## Libraries

In [ ]:
import pandas as pd
import os

## Step 1: Data Profiling
The goal here is to load a data sample and make an initial exploration. We will use a sample of 100,000 lines to get started.

### 1. Data Loading

In [ ]:
SOURCE_FILE = os.path.join('..', 'data', 'raw', 'train.csv')
SAMPLE_CHUNK_SIZE = 100000

In [ ]:
reader = pd.read_csv(SOURCE_FILE, chunksize=SAMPLE_CHUNK_SIZE)
df_sample = next(reader)
df_sample.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


### 2. Features Analysis

In [ ]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TRIP_ID       100000 non-null  int64  
 1   CALL_TYPE     100000 non-null  object 
 2   ORIGIN_CALL   20962 non-null   float64
 3   ORIGIN_STAND  49311 non-null   float64
 4   TAXI_ID       100000 non-null  int64  
 5   TIMESTAMP     100000 non-null  int64  
 6   DAY_TYPE      100000 non-null  object 
 7   MISSING_DATA  100000 non-null  bool   
 8   POLYLINE      100000 non-null  object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 6.2+ MB


#### 2.1 `TRIP_ID`

(String) It contains an unique identifier for each trip.

In [74]:
nunique_trip_id = df_sample['TRIP_ID'].nunique()

print(f'Number of unique TRIP_IDs: {nunique_trip_id}')

Number of unique TRIP_IDs: 99992


In [75]:

df_sample[df_sample['TRIP_ID'].duplicated(keep=False)]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
3519,1372702836620000080,B,NaN,16.0,20000080,1372702836,A,False,"[[-8.635275,41.193729],[-8.634519,41.192901]]"
3720,1372702836620000080,C,NaN,NaN,20000080,1372702836,A,False,"[[-8.62776,41.17554],[-8.62776,41.17554],[-8.6..."
21282,1373025987620000601,A,2002.0,NaN,20000601,1373025987,A,False,[]
21385,1373025987620000601,C,NaN,NaN,20000601,1373025987,A,False,"[[-8.604108,41.182569],[-8.604072,41.182605],[..."
33228,1373210896620000598,B,NaN,57.0,20000598,1373210896,A,False,"[[-8.610885,41.145525]]"
33350,1373210896620000598,C,NaN,NaN,20000598,1373210896,A,False,"[[-8.610813,41.14548],[-8.610732,41.145579],[-..."
51623,1373551710620000014,B,NaN,49.0,20000014,1373551710,A,False,"[[-8.587278,41.147325],[-8.587278,41.147325]]"
51966,1373551710620000014,C,NaN,NaN,20000014,1373551710,A,False,"[[-8.602938,41.179716],[-8.603937,41.179932],[..."
76527,1374014097620000337,C,NaN,NaN,20000337,1374014097,A,False,"[[-8.648451,41.164785]]"
76596,1374014097620000337,C,NaN,NaN,20000337,1374014097,A,False,"[[-8.648478,41.164758],[-8.648478,41.164758],[..."


#### 2.2 `CALL_TYPE`
(char) It identifies the way used to demand this service. It may contain one of three possible values:
* ‘A’ if this trip was dispatched from the central;
* ‘B’ if this trip was demanded directly to a taxi driver on a specific stand;
* ‘C’ otherwise (i.e. a trip demanded on a random street).

In [ ]:
df_sample['CALL_TYPE'].value_counts(normalize=True).sort_index()*100

CALL_TYPE
A    20.962
B    49.922
C    29.116
Name: proportion, dtype: float64

In [52]:
df_sample['CALL_TYPE'].value_counts().sort_index()

CALL_TYPE
A    20962
B    49922
C    29116
Name: count, dtype: int64

#### 2.3 `ORIGIN_CALL`

(integer) It contains an unique identifier for each phone number which was used to demand, at least, one service. It identifies the trip’s customer if CALL_TYPE=’A’. Otherwise, it assumes a NULL value.

In [46]:
df_sample['ORIGIN_CALL'].groupby(df_sample['CALL_TYPE']).agg(['count', 'nunique']) # ok

,count,nunique
CALL_TYPE,,
A,20962,8160
B,0,0
C,0,0


In [58]:
df_sample[df_sample['CALL_TYPE'] == 'A']['CALL_TYPE'].count() # ok, there's the same number of ORIGIN_CALL non-null values

np.int64(20962)

#### 2.4 `ORIGIN_STAND`

(integer): It contains an unique identifier for the taxi stand. It identifies the starting point of the trip if CALL_TYPE=’B’. Otherwise, it assumes a NULL value.

In [64]:
origin_stands_number = df_sample['ORIGIN_STAND'].count()
print(f'Number of non-null ORIGIN_STAND values: {origin_stands_number}')

Number of non-null ORIGIN_STAND values: 49311


In [51]:
df_sample['ORIGIN_STAND'].groupby(df_sample['CALL_TYPE']).agg(['count', 'nunique']) # ok

,count,nunique
CALL_TYPE,,
A,0,0
B,49311,62
C,0,0


In [65]:
b_call_type_number = df_sample[df_sample['CALL_TYPE'] == 'B']['CALL_TYPE'].count()
print(f'Number of CALL_TYPE B values: {b_call_type_number}')

Number of CALL_TYPE B values: 49922


In [68]:
diff_origin_stands = b_call_type_number - origin_stands_number
print(f'Difference between CALL_TYPE B count and non-null ORIGIN_STAND count: {diff_origin_stands}')
print('This difference should be zero if every CALL_TYPE B has a corresponding ORIGIN_STAND value.')


Difference between CALL_TYPE B count and non-null ORIGIN_STAND count: 611
This difference should be zero if every CALL_TYPE B has a corresponding ORIGIN_STAND value.


#### 2.5 `TAXI_ID`

(integer): It contains an unique identifier for the taxi driver that performed each trip.

In [69]:
df_sample['TAXI_ID'].nunique()

430

#### 2.6 `TIMESTAMP`

(integer) Unix Timestamp (in seconds). It identifies the trip’s start.

In [70]:
df_sample['TIMESTAMP'].nunique()

96801

#### 2.7 `DAYTYPE`

(char) It identifies the daytype of the trip’s start. It assumes one of three possible values:
* ‘B’ if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.);
* ‘C’ if the trip started on a day before a type-B day;
* ‘A’ otherwise (i.e. a normal day, workday or weekend).

In [71]:
df_sample['DAY_TYPE'].value_counts().sort_index()

DAY_TYPE
A    100000
Name: count, dtype: int64

#### 2.8 `MISSING_DATA`

(Boolean) It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing.

In [29]:
df_sample['MISSING_DATA'].value_counts()

MISSING_DATA
False    100000
Name: count, dtype: int64

#### 2.9 `POLYLINE`

(String): It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for each 15 seconds of trip. The last list item corresponds to the trip’s destination while the first one represents its start.

In [ ]:
df_sample['POLYLINE'][0]

'[[-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489]]'

In [33]:
type(df_sample['POLYLINE'][0])

str

### 3. Results